# 3 Sample Tests Power over Increasing Epsilon

These are same useful functions to import. Since we are calculating the statistical power over all the tests for all the simulations, we can just use a wild card import from the respective modules

In [1]:
import sys, os
from joblib import Parallel, delayed

import numpy as np
import matplotlib.pyplot as plt

from benchmarks import power_3samp_epsweight
from mgc.independence import Dcorr, Hsic, MGC
from mgc.sims import gaussian_3samp

sys.path.append(os.path.realpath('..'))

In [2]:
import seaborn as sns
sns.set(color_codes=True, style='white', context='talk', font_scale=2)
sns.set_palette(sns.color_palette("Set1"))

In [3]:
from statsmodels.multivariate.manova import MANOVA


class PyManova:
    r"""
    Wrapper of statsmodels MANOVA
    """

    def _statistic(self, x, y):
        r"""
        Helper function to calculate the test statistic
        """
        stat = MANOVA(x, y).mv_test().results['x0']['stat'].values[1, 0]
        self.stat = stat

        return stat

In [4]:
from rpy2.robjects import Formula, numpy2ri
from rpy2.robjects.packages import importr


class Manova:
    r"""
    Wrapper of R MANOVA
    """
    def __init__(self):
        self.stats = importr('stats')
        self.r_base = importr('base')
        
        numpy2ri.activate()

        self.formula = Formula('X ~ Y')
        self.env = self.formula.environment

    def _statistic(self, x, y):
        r"""
        Helper function to calculate the test statistic
        """
        self.env['Y'] = y
        self.env['X'] = x

        stat = self.r_base.summary(self.stats.manova(self.formula), test="Pillai")[3][4]

        return stat

These are some constants that are used in this notebook. If running these notebook, please only manipulate these constants if you are not running more tests. They define the epsilons (distance from 0 for the center of each gaussian cluster) tested upon and the number of replications. The simulations tested over and the independence tests tested over are defined also.

In [5]:
MAX_EPSILON = 1
STEP_SIZE = 0.05
EPSILONS = np.arange(0, MAX_EPSILON + STEP_SIZE, STEP_SIZE)
WEIGHTS = EPSILONS
POWER_REPS = 5

In [6]:
cases = [
    1,
    2,
    3,
    4,
    5
]

tests = [
    MGC,
    Dcorr,
    Hsic,
    PyManova,
    Manova
]

The following function calculates the estimated power ``POWER_REPS`` number off times and averages them. It does this iterating over the number of sample sizes.

**Note: We only recommend running this code if running the next 2 cells ONCE to generate the csv files used to visualize the plots. This code takes a very long time to run and if running, we recommend using a machine with many cores.**

In [7]:
def estimate_power(case, test):
    if case in [4, 5]:
        est_power = np.array([np.mean([power_3samp_epsweight(test, case=case, weight=i) for _ in range(POWER_REPS)])
                              for i in WEIGHTS])
    else:
        est_power = np.array([np.mean([power_3samp_epsweight(test, case=case, epsilon=i) for _ in range(POWER_REPS)])
                              for i in EPSILONS])
    np.savetxt('../benchmarks/3samp_vs_epsilon/{}_{}.csv'.format(case, test.__name__),
               est_power, delimiter=',')
    
    return est_power

In [8]:
outputs = Parallel(n_jobs=-1, verbose=100)(
    [delayed(estimate_power)(case, test) for case in cases for test in tests]
)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Pickling array (shape=(21,), dtype=float64).
Pickling array (shape=(21,), dtype=float64).
Pickling array (shape=(21,), dtype=float64).
Pickling array (shape=(21,), dtype=float64).
Pickling array (shape=(21,), dtype=float64).
Pickling array (shape=(21,), dtype=float64).
Pickling array (shape=(21,), dtype=float64).
Pickling array (shape=(21,), dtype=float64).
Pickling array (shape=(21,), dtype=float64).
Pickling array (shape=(21,), dtype=float64).
Pickling array (shape=(21,), dtype=float64).
Pickling array (shape=(21,), dtype=float64).
Pickling array (shape=(21,), dtype=float64).
Pickling array (shape=(21,), dtype=float64).
Pickling array (shape=(21,), dtype=float64).
Pickling array (shape=(21,), dtype=float64).
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   44.5s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   44.5s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   44.5s
[Parallel(n_jobs=-

KeyboardInterrupt: 

The following code loops over each saved independence test file and generates absolute power curves for each test and for each simulation modality.

In [ ]:
def plot_power():
    fig, ax = plt.subplots(nrows=2, ncols=5, figsize=(14,8))
    
    sim_title = [
        "Three Equal",
        "Two Equal",
        "None Equal",
        "One Not Gaussian",
        "None Gaussian"
    ]
    
    for i, row in enumerate(ax):
        for j, col in enumerate(row):
            if i == 0:
                sims = gaussian_3samp(100, epsilon=4, weight=0.7, case=cases[j])
                
                for sim in sims:
                    x, y = np.hsplit(sim, 2)
                    col.scatter(x, y)

#                     col.set_xlim(-4, 4)
                    col.set_xticks([])
#                     col.set_ylim(-4, 4)
                    col.set_yticks([])
                    col.set_title(sim_title[j])
            else:
                case = cases[j]

                for test in tests:
                    power = np.genfromtxt('../benchmarks/3samp_vs_epsilon/{}_{}.csv'.format(case, test.__name__),
                                          delimiter=',')

                    custom_color = {
                        "Dcorr" : "k-",
                        "Hsic" : "g-",
                        "MGC" : "r-",
                    }
                    if test.__name__ in custom_color.keys():
                        if test.__name__ == "MGC":
                            col.plot(EPSILONS, power, custom_color[test.__name__], label=test.__name__, lw=5)
                        else:
                            col.plot(EPSILONS, power, custom_color[test.__name__], label=test.__name__, lw=2)
                    else:
                        col.plot(EPSILONS, power, label=test.__name__, lw=1)
                    col.set_xticks([EPSILONS[0], EPSILONS[-1]])
                    col.set_ylim(0, 1.05)
                    col.set_yticks([])
                    if j == 0:
                        col.set_yticks([0, 1])
    
    fig.text(0.5, 0.01, 'Increasing Epsilon', ha='center')
    fig.text(0.5, 0.01, 'Increasing Weight', ha='center')
    fig.text(0.06, 0.3, 'Power', va='center', rotation='vertical')
    
    leg = plt.legend(bbox_to_anchor=(0.5, 0.02), bbox_transform=plt.gcf().transFigure,
                     ncol=3, loc='upper center')
    leg.get_frame().set_linewidth(0.0)
    for legobj in leg.legendHandles:
        legobj.set_linewidth(5.0)
    plt.subplots_adjust(hspace=.20)
    plt.savefig('../benchmarks/figs/3samp_power_epsilon.pdf', transparent=True, bbox_inches='tight')

In [ ]:
plot_power()